In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster

In [ ]:
# Configurar Dask para usar GPUs
cluster = LocalCUDACluster()
client = Client(cluster)

In [ ]:
# Definir un modelo simple de PyTorch
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(10, 50)
        self.fc2 = nn.Linear(50, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
# Datos de ejemplo
X = torch.randn(1000, 10)
y = torch.randn(1000, 1)

In [ ]:
# Crear el modelo
model = SimpleNet()
model = nn.DataParallel(model)  # Usar DataParallel para múltiples GPUs (se ejecuta bien en una sola GPU)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
# Función de entrenamiento
def train_model(X, y, model, criterion, optimizer, epochs=5):
    X = X.cuda()
    y = y.cuda()
    model = model.cuda()
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(X)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
    return model.cpu()  # Mover el modelo de nuevo a la CPU antes de devolverlo

In [ ]:
# Mover los datos a la CPU antes de enviarlos a Dask
X_cpu = X.cpu()
y_cpu = y.cpu()
model_cpu = model.cpu()

In [ ]:
# Paralelizar el entrenamiento con Dask
# Nota: los datos y el modelo se mueven a la GPU dentro de la función de entrenamiento
future = client.submit(train_model, X_cpu, y_cpu, model_cpu, criterion, optimizer)
wait(future)
trained_model = future.result()

In [ ]:
# Verificar el modelo entrenado
print(trained_model)

In [ ]:
# Validar el modelo entrenado
# Datos de prueba
X_test = torch.randn(200, 10).cuda()
y_test = torch.randn(200, 1).cuda()

In [ ]:
# Mover el modelo entrenado de nuevo a la GPU para evaluación
trained_model = trained_model.cuda()

In [ ]:
# Realizar predicciones
trained_model.eval()
with torch.no_grad():
    predictions = trained_model(X_test)

In [ ]:
# Calcular el error cuadrático medio
criterion = nn.MSELoss()
mse = criterion(predictions, y_test)
print(f'Mean Squared Error: {mse.item()}')